In [1]:
import sys
sys.path.insert(0, '/scratch/cinthiasouza/mv-text-summarizer')

import pandas as pd
import re

In [2]:
from src import loader
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result/{}/{}_*.csv"

In [3]:
def clean_text(text):
    
    text = re.sub(r"[^a-zA-Z]+", ' ', text)
    return text

In [4]:
def remove_unicode(text):

    text = str(text).encode("ascii", "ignore")
    text = text.decode()

    return text

In [5]:
def num_sentences(grouped_df):

    lenght = {'id': [], 'lenght': []}

    for idx, group in grouped_df:
        lenght['id'].append(idx)
        lenght['lenght'].append(group.shape[0])
        
    return  pd.DataFrame(lenght)

In [61]:
features_intro = pd.read_csv('features_intro.csv')

In [6]:
features_intro = pd.read_csv('features_intro.csv')
features_mat = pd.read_csv('features_mat.csv')
features_conc = pd.read_csv('features_conc.csv')

scores_intro = pd.read_csv('scores_intro.csv')
scores_mat = pd.read_csv('scores_mat.csv')
scores_conc = pd.read_csv('scores_conc.csv')

In [7]:
features_intro['rouge_1'] = scores_intro['rouge_1']
features_intro['rouge_2'] = scores_intro['rouge_2']

In [8]:
features_mat['rouge_1'] = scores_mat['rouge_1']
features_mat['rouge_2'] = scores_mat['rouge_2']

In [9]:
features_conc['rouge_1'] = scores_conc['rouge_1']
features_conc['rouge_2'] = scores_conc['rouge_2']

In [10]:
summ_items = pd.read_csv("indices_summ.csv")['summ']

In [12]:
def clean_df(df):
    
    df['sentences'] = df['sentences'].apply(remove_unicode)
    df = df[~df['sentences'].str.isdigit()]

    df = df.loc[df['sentences'] != " "].reset_index(drop=True)
    df = df.loc[df['sentences'] != ""].reset_index(drop=True)

    grouped_df = df.groupby('articles')

    lenght_df = num_sentences(grouped_df)
    lenght_df = lenght_df.loc[lenght_df['lenght'] >= 6]

    train_articles_id = lenght_df['id'].tolist()

    df = df.loc[df['articles'].isin(train_articles_id)]

    return df, lenght_df

In [13]:
clean_intro, lenght_intro = clean_df(features_intro)
clean_mat, lenght_mat = clean_df(features_mat)
clean_conc, lenght_conc = clean_df(features_conc)

In [15]:
import random

In [16]:
intro = pd.unique(clean_intro['articles'])
mat = pd.unique(clean_mat['articles'])
conc = pd.unique(clean_conc['articles'])

comuns = list((set(intro) & set(mat)) & set(conc))

valid_len = int(len(comuns)*0.2)
summ_items = random.sample(comuns, valid_len)

df = pd.DataFrame({'summ': summ_items})
df.to_csv("dataset/indices_summ.csv", index=False)

In [ ]:
len(summ_items)

In [ ]:
clean_intro.to_csv('dataset/dataset_introduction.csv', index=False)
clean_mat.to_csv('dataset/dataset_materials.csv', index=False)
clean_conc.to_csv('dataset/dataset_conclusion.csv', index=False)